## Open AI function to send prompt to gpt 

In [24]:
import openai
def ask_openai(text, api_key):
    openai.api_key = api_key

    # Ensure you're using the correct engine model. Update as necessary.
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Update to the specific model you intend to use, like "gpt-4" when available.
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Extract Report No,Patient Name,Date,Age,HGB,HCT,PLT,WBC,RBC,ALT from the following medical text, and mentioned result with units. also add  specimen id as report no,and DONT WRITE HEADING for Report No,Patient Name,Date,Age,HGB,HCT,PLT,WBC,RBC,ALT and also any other text and write values NULL if theres no result,AND USE - IN DATE NOT COMMA, I NEED ANSWER FOR ONLY 10 ATTRIBUTES WHICH I GAVE U :\n\n{text}"}
        ],
        max_tokens=250
    )

    # Depending on the structure of the response, adjust how you access the generated text.
    return response['choices'][0]['message']['content'].strip()

## OCR function to get text from lab reports

In [25]:
import pytesseract
from pdf2image import convert_from_path

# If you have installed Tesseract-OCR in a specific directory, you might need to specify the path to the executable
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Adjust the path based on your installation

def extract_text_from_pdf_with_ocr(pdf_path):
    # Convert PDF to a list of images (one image per page)
    images = convert_from_path(pdf_path, dpi=300)  # dpi can be adjusted based on the quality of the scanned document
    
    text = ''
    # Iterate through the images and apply OCR to each
    for image in images:
        text += pytesseract.image_to_string(image)
    
    return text

## API KEY IS CONFIDENTIAL MAKE SURE TO REMOVE IT WHEN U UPLOAD ON GITHUB

In [33]:
pdf_path = 'bloodreport3.pdf'  # Specify the path to your PDF
extracted_text = extract_text_from_pdf_with_ocr(pdf_path)
api_key= "api-key"
result = ask_openai(extracted_text, api_key)
print(result)

001IDON23012756, HASSAN, December-1-2023, NULL, 15.5 g/dL, 48.2 %, 369 x10³/µL, 8.53 x10³/µL, 5.52 x10⁶/µL, NULL


## Function to create dataset from our lab report which was give to open ai for to give correct results

In [34]:
import pandas as pd

def append_lab_report_from_csv_string(csv_string, file_path):
    # Split the CSV string into a list
    report_data = csv_string.split(', ')

    # Prepare a DataFrame with the correct columns based on your report format
    columns = ["Report No", "Patient Name", "Date", "Age", "HGB", "HCT", "PLT", "WBC", "RBC", "ALT"]
    df_new_entry = pd.DataFrame([report_data], columns=columns)

    # Load existing data or initialize a new DataFrame if the file does not exist
    try:
        df_existing = pd.read_csv(file_path)
    except FileNotFoundError:
        df_existing = pd.DataFrame(columns=columns)

    # Check if the new report number is already present in the existing data
    if df_new_entry['Report No'].iloc[0] in df_existing['Report No'].values:
        return "Report already present."

    # Append new data if report number is not found
    df_combined = pd.concat([df_existing, df_new_entry])
    df_combined.to_csv(file_path, index=False)
    return "Data appended successfully."

# Example usage of the function
file_path = 'Lab_Reports.csv'  # Specify the file path for the CSV
csv_input = result  # Example CSV input

# Call the function and print the response
response = append_lab_report_from_csv_string(csv_input, file_path)
print(response)


Report already present.


In [38]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

# Load the dataset
df = pd.read_csv('Lab_Reports.csv')  # Adjust the path as necessary

# Initialize the NetworkX graph
G = nx.DiGraph()

# Add nodes and edges for each report
for index, row in df.iterrows():
    report_node = f"Report: {row['Report No']} ({row['Date']})"
    patient_node = f"Patient: {row['Patient Name']} ({row['Date']})"

    # Add report and patient nodes
    G.add_node(report_node, title=report_node, color="orange", size=25)
    G.add_node(patient_node, title=patient_node, color="lightblue", size=30)
    
    # Connect patient to report
    G.add_edge(patient_node, report_node, title="Report Details")

    # Create nodes for each test result and connect to the report
    hgb_node = f"HGB {row['HGB']} ({row['Date']})"
    hct_node = f"HCT {row['HCT']} ({row['Date']})"
    plt_node = f"PLT {row['PLT']} ({row['Date']})"
    wbc_node = f"WBC {row['WBC']} ({row['Date']})"
    rbc_node = f"RBC {row['RBC']} ({row['Date']})"
    alt_node = f"ALT {row['ALT']} ({row['Date']})"

    # Adding nodes for each test with specific colors and sizes
    G.add_node(hgb_node, title=hgb_node, color="green", size=20)
    G.add_node(hct_node, title=hct_node, color="red", size=20)
    G.add_node(plt_node, title=plt_node, color="yellow", size=20)
    G.add_node(wbc_node, title=wbc_node, color="purple", size=20)
    G.add_node(rbc_node, title=rbc_node, color="blue", size=20)
    G.add_node(alt_node, title=alt_node, color="pink", size=20)

    # Connect these nodes to the report node
    G.add_edge(report_node, hgb_node, title="HGB Level")
    G.add_edge(report_node, hct_node, title="HCT Level")
    G.add_edge(report_node, plt_node, title="PLT Count")
    G.add_edge(report_node, wbc_node, title="WBC Count")
    G.add_edge(report_node, rbc_node, title="RBC Count")
    G.add_edge(report_node, alt_node, title="ALT Level")

# Initialize PyVis network
nt = Network('1000px', '2000px', notebook=True)  # Set notebook=True if using in Jupyter Notebooks
nt.from_nx(G)
nt.show_buttons(filter_=['physics'])  # Allow interactive control over physics settings
nt.show("lab_report_visualization.html")


lab_report_visualization.html
